In [ ]:
################
#achieving Media data#

In [ ]:
def media_feature(df):
    subreddits = ['CoronavirusUS','CoronavirusUK','CoronavirusNewYork','CoronavirusCA','Talk about Coronavirus in Michigan ','CoronaVirusPA','CoronavirusIllinois']
    pre = []
    concerns = []
    total = []
    for i in subreddits:
        pre.append(0)
        concerns.append(0)
        total.append(0)
    for i,row in df.iterrows():
        k = subreddits.index(row["subreddit"])
        if(row["precaution"] == 1):
            pre[k] += 1
        if(row["concerns"] == 1):
            concerns[k] += 1
        total[k] += 1
    return pre,concerns,total

In [ ]:
###############

In [ ]:
#SIR#